In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a='/content/drive/MyDrive/SpotifyFeatures.csv'

In [ ]:
df=pd.read_csv(a)

In [ ]:
df.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [ ]:
df.isnull().sum()

genre               0
artist_name         0
track_name          0
track_id            0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
dtype: int64

In [ ]:
df.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [ ]:
corpus=df.genre.unique()
corpus

array(['Movie', 'R&B', 'A Capella', 'Alternative', 'Country', 'Dance',
       'Electronic', 'Anime', 'Folk', 'Blues', 'Opera', 'Hip-Hop',
       "Children's Music", 'Children’s Music', 'Rap', 'Indie',
       'Classical', 'Pop', 'Reggae', 'Reggaeton', 'Jazz', 'Rock', 'Ska',
       'Comedy', 'Soul', 'Soundtrack', 'World'], dtype=object)

Data pre processing

Tf-Idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(corpus)

In [ ]:
genre_df = pd.DataFrame(X.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
tfidf.get_feature_names_out()

array(['alternative', 'anime', 'blues', 'capella', 'children',
       'classical', 'comedy', 'country', 'dance', 'electronic', 'folk',
       'hip', 'hop', 'indie', 'jazz', 'movie', 'music', 'opera', 'pop',
       'rap', 'reggae', 'reggaeton', 'rock', 'ska', 'soul', 'soundtrack',
       'world'], dtype=object)

Min Max

In [ ]:
scaled_features = MinMaxScaler().fit_transform([
  df['acousticness'].values,
  df['danceability'].values,
  df['duration_ms'].values,
  df['energy'].values,
  df['instrumentalness'].values,
  df['liveness'].values,
  df['loudness'].values,
  df['speechiness'].values,
  df['tempo'].values,
  df['valence'].values,
  ])

sentiment analysis

In [ ]:
from textblob import TextBlob

In [ ]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [ ]:
sa_df=sentiment_analysis(df, "track_name")

In [ ]:
polar_ohe=pd.get_dummies(df.polarity)
sub_ohe=pd.get_dummies(df.subjectivity)
key_OHE = pd.get_dummies(df.key)

In [ ]:
genre_df.isnull().sum()

genre|alternative    0
genre|anime          0
genre|blues          0
genre|capella        0
genre|children       0
genre|classical      0
genre|comedy         0
genre|country        0
genre|dance          0
genre|electronic     0
genre|folk           0
genre|hip            0
genre|hop            0
genre|indie          0
genre|jazz           0
genre|movie          0
genre|music          0
genre|opera          0
genre|pop            0
genre|rap            0
genre|reggae         0
genre|reggaeton      0
genre|rock           0
genre|ska            0
genre|soul           0
genre|soundtrack     0
genre|world          0
dtype: int64

In [ ]:
df[['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence']] = scaled_features.T

In [ ]:
df = df.drop('genre',axis = 1)
df = df.drop('artist_name', axis = 1)
df = df.drop('track_name', axis = 1)
df = df.drop('popularity',axis = 1)
df = df.drop('key', axis = 1)
df = df.drop('mode', axis = 1)
df = df.drop('time_signature', axis = 1)

In [ ]:
df = df.join(genre_df,how='inner')
df = df.join(key_OHE)
df=df.join(polar_ohe)
df=df.join(sub_ohe)

In [ ]:
df.head()

,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,F,F#,G,G#,Negative,Neutral,Positive,high,low,medium
0,0BRjO6ga9RKCKjfDqeFgWV,0.000025,0.000022,1.0,0.000028,0.000018,0.000022,0.0,0.000019,0.001699,...,0,0,0,0,0,1,0,0,1,0
1,0BjC1NfoEOOusryehmNudP,0.000042,0.000045,1.0,0.000046,0.000040,0.000042,0.0,0.000041,0.001307,...,0,1,0,0,0,1,0,0,1,0
2,0CoSDzoNIKCRs124s9uTVy,0.000087,0.000085,1.0,0.000082,0.000082,0.000082,0.0,0.000082,0.000666,...,0,0,0,0,1,0,0,1,0,0
3,0Gc6TVm52BwZD07Ki6tIvf,0.000084,0.000081,1.0,0.000082,0.000080,0.000081,0.0,0.000080,0.001207,...,0,0,0,0,0,1,0,0,1,0
4,0IuslXpMROHdEPvSl1fTQK,0.000267,0.000260,1.0,0.000259,0.000257,0.000258,0.0,0.000256,0.001957,...,1,0,0,0,0,1,0,0,1,0


In [ ]:
df.isnull().sum()

track_id             0
acousticness         0
danceability         0
duration_ms          0
energy               0
instrumentalness     0
liveness             0
loudness             0
speechiness          0
tempo                0
valence              0
subjectivity         0
polarity             0
genre|alternative    0
genre|anime          0
genre|blues          0
genre|capella        0
genre|children       0
genre|classical      0
genre|comedy         0
genre|country        0
genre|dance          0
genre|electronic     0
genre|folk           0
genre|hip            0
genre|hop            0
genre|indie          0
genre|jazz           0
genre|movie          0
genre|music          0
genre|opera          0
genre|pop            0
genre|rap            0
genre|reggae         0
genre|reggaeton      0
genre|rock           0
genre|ska            0
genre|soul           0
genre|soundtrack     0
genre|world          0
A                    0
A#                   0
B                    0
C          

In [ ]:
df.columns

Index(['track_id', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'subjectivity', 'polarity', 'genre|alternative',
       'genre|anime', 'genre|blues', 'genre|capella', 'genre|children',
       'genre|classical', 'genre|comedy', 'genre|country', 'genre|dance',
       'genre|electronic', 'genre|folk', 'genre|hip', 'genre|hop',
       'genre|indie', 'genre|jazz', 'genre|movie', 'genre|music',
       'genre|opera', 'genre|pop', 'genre|rap', 'genre|reggae',
       'genre|reggaeton', 'genre|rock', 'genre|ska', 'genre|soul',
       'genre|soundtrack', 'genre|world', 'A', 'A#', 'B', 'C', 'C#', 'D', 'D#',
       'E', 'F', 'F#', 'G', 'G#', 'Negative', 'Neutral', 'Positive', 'high',
       'low', 'medium'],
      dtype='object')

In [ ]:
df=df.drop(['subjectivity','polarity'],axis=1)

dataset is pre processed 
final dataset is made

#getting playlist from api

In [ ]:
pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
cid = 'b726d60495384395991dfa92dc0ffec8'
secret = 'd932a84ab95c4d4f89651d2b4a1d62dc'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
def call_playlist(creator, playlist_id):
    
    #step1

    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    #Step 3
        
    return playlist_df

In [ ]:
playlist_df=call_playlist('spotify','0xk151LNOx9rvvfbQy5jJn')

In [ ]:
playlist_df.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Selena Gomez,Stars Dance (Bonus Track Version),Save The Day,594fJDaRhC2bcHilqoc8U4,0.666,0.836,0,-3.599,1,0.0360,0.000022,0.132,0.531,119.995,232973,4
1,Various Artists,Four More Shots Please! (Original Series Sound...,Yaara Teri Yaari,5DUDLaokWUsRbU8XLuQM2y,0.491,0.665,7,-5.335,1,0.0386,0.000004,0.120,0.266,160.030,207000,4
2,Asees Kaur,"Hui Malang (From ""Malang - Unleash The Madness"")","Hui Malang (From ""Malang - Unleash The Madness"")",5bAed2aRbQbjDZF9qkNSA2,0.633,0.904,7,-5.090,0,0.0564,0.000043,0.122,0.259,124.008,192097,4
3,Witt Lowry,Into Your Arms (feat. Ava Max),Into Your Arms (feat. Ava Max),0b11D9D0hMOYCIMN3OKreM,0.504,0.788,3,-4.113,0,0.2150,0.000000,0.363,0.228,170.039,186022,4
4,Harrdy Sandhu,Backbone,Backbone,4t77OsiudWy4timV32lPa2,0.501,0.828,3,-2.495,0,0.1140,0.000000,0.135,0.483,84.659,175059,4


In [ ]:
a=playlist_df.columns

In [ ]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    '''
    Summarize a user's playlist into a single vector
    ---
    Input: 
    complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
    playlist_df (pandas dataframe): playlist dataframe
        
    Output: 
    complete_feature_set_playlist_final (pandas series): single vector feature that summarizes the playlist
    complete_feature_set_nonplaylist (pandas dataframe): 
    '''
    
    # Find song features in the playlist
    complete_feature_set_playlist = df[df['track_id'].isin(playlist_df['track_id'].values)]
    # Find all non-playlist song features
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['track_id'].isin(playlist_df['track_id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = 'track_id')
    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
a,b=generate_playlist_feature(df, playlist_df)

In [ ]:
a

In [ ]:
b.head()

,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,F,F#,G,G#,Negative,Neutral,Positive,high,low,medium
0,0BRjO6ga9RKCKjfDqeFgWV,0.000025,0.000022,1.0,0.000028,0.000018,0.000022,0.0,0.000019,0.001699,...,0,0,0,0,0,1,0,0,1,0
1,0BjC1NfoEOOusryehmNudP,0.000042,0.000045,1.0,0.000046,0.000040,0.000042,0.0,0.000041,0.001307,...,0,1,0,0,0,1,0,0,1,0
2,0CoSDzoNIKCRs124s9uTVy,0.000087,0.000085,1.0,0.000082,0.000082,0.000082,0.0,0.000082,0.000666,...,0,0,0,0,1,0,0,1,0,0
3,0Gc6TVm52BwZD07Ki6tIvf,0.000084,0.000081,1.0,0.000082,0.000080,0.000081,0.0,0.000080,0.001207,...,0,0,0,0,0,1,0,0,1,0
4,0IuslXpMROHdEPvSl1fTQK,0.000267,0.000260,1.0,0.000259,0.000257,0.000258,0.0,0.000256,0.001957,...,1,0,0,0,0,1,0,0,1,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def generate_playlist_recos(spotify_df, features, nonplaylist_features):
    '''
    Generated recommendation based on songs in aspecific playlist.
    ---
    Input: 
    df (pandas dataframe): spotify dataframe
    features (pandas series): summarized playlist feature (single vector)
    nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Output: 
    non_playlist_df_top_40: Top 40 recommendations for that playlist
    '''
    
    non_playlist_df = df[df['track_id'].isin(nonplaylist_features['track_id'].values)]
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('track_id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [ ]:
c=generate_playlist_recos(playlist_df, a, b)

In [ ]:
c

,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,F#,G,G#,Negative,Neutral,Positive,high,low,medium,sim
0,0BRjO6ga9RKCKjfDqeFgWV,0.000025,0.000022,1.0,0.000028,0.000018,0.000022,0.0,0.000019,0.001699,...,0,0,0,0,1,0,0,1,0,0.0
14,0pXwl2CRP5awxHsF9eET3L,0.000093,0.000090,1.0,0.000088,0.000093,0.000088,0.0,0.000087,0.000809,...,0,1,0,0,0,1,1,0,0,0.0
25,16UTNMhXjeJa8IG9uBijxZ,0.000077,0.000077,1.0,0.000075,0.000074,0.000075,0.0,0.000077,0.000494,...,0,0,0,0,1,0,0,1,0,0.0
24,15nBPcHoH2fZmRiFd2ZhMh,0.000044,0.000043,1.0,0.000042,0.000040,0.000041,0.0,0.000041,0.000902,...,0,0,0,0,1,0,0,1,0,0.0
23,15CpJP0LXchBUjpB8RKh8m,0.000156,0.000157,1.0,0.000157,0.000147,0.000150,0.0,0.000147,0.001756,...,0,0,0,0,1,0,0,1,0,0.0
22,14K25Ks5fdHjHfpIYOTc4y,0.000046,0.000046,1.0,0.000046,0.000042,0.000043,0.0,0.000042,0.000833,...,0,0,0,0,1,0,0,1,0,0.0
21,148uA2Y78xfxDiCNjgZA0v,0.000070,0.000072,1.0,0.000070,0.000067,0.000068,0.0,0.000068,0.000765,...,0,0,0,0,1,0,0,1,0,0.0
20,12hJJLMMbJDK0F9I3g5SLv,0.000036,0.000033,1.0,0.000034,0.000032,0.000032,0.0,0.000032,0.000394,...,0,0,0,0,1,0,1,0,0,0.0
19,12ZjoNweObu1k9rK4SOSdL,0.000026,0.000028,1.0,0.000029,0.000025,0.000026,0.0,0.000026,0.000913,...,0,0,0,0,1,0,0,1,0,0.0
18,113pHPsGwyeklEjLhnB3ZT,0.000047,0.000047,1.0,0.000047,0.000045,0.000046,0.0,0.000045,0.000413,...,0,0,0,0,1,0,0,1,0,0.0
